In [1]:
import numpy as np
import pandas as pd
import recommenders as rec
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit

%matplotlib inline

In [2]:
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
df_city_checkin=pd.read_csv("../data/checkin/lasvegas.csv",converters={'categories':string_to_array})
users_id=df_city_checkin['user_id'].drop_duplicates().reset_index(drop=True)

# USG uses unique checkins cij = 1 or cij = 0
df_city_checkin=df_city_checkin.drop_duplicates(subset=['user_id','business_id']).reset_index(drop=True)

In [3]:
df_city_checkin.head()

,user_id,business_id,date,latitude,longitude,categories,subarea_id
0,46TNf-5T5EcNFRJPYv_nrw,ujmEBvifdJM6h6RLv4wQIg,2015-08-19 16:39:53,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
1,56iEnLi8jR--2ranjPSQ4w,ujmEBvifdJM6h6RLv4wQIg,2013-08-04 10:55:04,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
2,UDKqdVT0FrpL19shSOqgow,ujmEBvifdJM6h6RLv4wQIg,2017-03-15 23:27:38,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
3,Nw6Jza_cb0aqd4PT769Vzw,ujmEBvifdJM6h6RLv4wQIg,2013-01-04 22:04:02,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0
4,3b5EsiTkqgdbxPXz7LfBpA,ujmEBvifdJM6h6RLv4wQIg,2014-08-11 23:57:54,36.215546,-115.248798,"[Fitness & Instruction, Doctors, Gyms, Emergen...",1415.0


$CosineSimilarity = cos(\pmb x, \pmb y) = \frac {\pmb x \cdot \pmb y}{||\pmb x|| \cdot ||\pmb y||}$

In [4]:
def user_similarity(user_checkin_1,user_checkin_2):
    common_checkin=pd.merge(user_checkin_1,user_checkin_2,on='business_id')
    return common_checkin.date_x.count()/(math.sqrt(user_checkin_1.date.count())
                                   *math.sqrt(user_checkin_2.date.count()))

user_similarity(df_city_checkin[df_city_checkin.user_id=='46TNf-5T5EcNFRJPYv_nrw'],
                df_city_checkin[df_city_checkin.user_id=='56iEnLi8jR--2ranjPSQ4w'])

0.02973505167250263

$\#df\_user\_similarity=\frac{\#usuarios\cdot (\#usuarios-1)}{2}=\sum_{i=1}^{\#usuarios}{i}$

In [ ]:
df_user_similarity=pd.DataFrame(columns=['user_id_1','user_id_2','similarity'])
i=0
start = timeit.default_timer()
# loop sobre os ids dos usuarios
for index_1,user_id_1 in users_id.iteritems():
    # lista de checkins do usuario
    df_user_checkin_1=df_city_checkin[df_city_checkin.user_id==user_id_1]
    # Exclui o proprio usuario
    for index_2,user_id_2 in users_id.drop(index_1).iteritems():
#         if df_city_checkin[(df_user_similarity.user_id_1==user_id_1 | df_user_similarity.user_id_1==user_id_2) &
#                             (df_user_similarity.user_id_2==user_id_1 | df_user_similarity.user_id_2==user_id_2)]:
#             pass

        if len(df_user_similarity[((df_user_similarity.user_id_1==user_id_1) | (df_user_similarity.user_id_1==user_id_2)) &
                        ((df_user_similarity.user_id_2==user_id_1) | (df_user_similarity.user_id_2==user_id_2))])!=0:
            continue
            
        df_user_checkin_2=df_city_checkin[df_city_checkin.user_id==user_id_2]
        sim=user_similarity(df_user_checkin_1,df_user_checkin_2)
        df_user_similarity=df_user_similarity.append({'user_id_1':user_id_1,'user_id_2':user_id_2,'similarity':sim},ignore_index=True)
        if i % 100000 == 0:
            print("user",i)
        i=i+1
#         print(user_id)
#         df_user_checkin=df_city_checkin[df_city_checkin[.index]==user_id]
#         print(df_user_checkin)
stop = timeit.default_timer()
print('Time:', stop - start)
df_user_similarity

user 0


In [81]:
def user_cf(user_id,user_checkin,business_id,df_checkin):
    df_business_checkin=df_checkin[df_checkin['business_id']==business_id]
    df_checkin=df_checkin[df_city_checkin['user_id']!=user_id]
    return df_checkin.apply(lambda checkin: user_similarity(user_checkin,df_checkin[df_checkin['user_id']==checkin['user_id']]),axis=1)

user_cf('46TNf-5T5EcNFRJPYv_nrw',df_city_checkin[df_city_checkin.user_id=='46TNf-5T5EcNFRJPYv_nrw'],'ujmEBvifdJM6h6RLv4wQIg',df_city_checkin)

KeyboardInterrupt: 

In [40]:
for index,user_id in users_id.iteritems():
    print(user_id)
    df_user_checkin=df_city_checkin[df_city_checkin[.index]==user_id]
    print(df_user_checkin)
    break

46TNf-5T5EcNFRJPYv_nrw
                  user_id             business_id                 date  \
0  46TNf-5T5EcNFRJPYv_nrw  ujmEBvifdJM6h6RLv4wQIg  2015-08-19 16:39:53   

    latitude   longitude                                         categories  \
0  36.215546 -115.248798  [Fitness & Instruction, Doctors, Gyms, Emergen...   

   subarea_id  
0      1415.0  


In [10]:
i=0
df_city_checkin[df_city_checkin.index=='46TNf-5T5EcNFRJPYv_nrw']
for index,checkin in df_city_checkin.iteritems():
    print(index)
    print(checkin)
    if i%22==0:
        print(i)
        break
    i=i+1

business_id
user_id
46TNf-5T5EcNFRJPYv_nrw    ujmEBvifdJM6h6RLv4wQIg
56iEnLi8jR--2ranjPSQ4w    ujmEBvifdJM6h6RLv4wQIg
UDKqdVT0FrpL19shSOqgow    ujmEBvifdJM6h6RLv4wQIg
Nw6Jza_cb0aqd4PT769Vzw    ujmEBvifdJM6h6RLv4wQIg
3b5EsiTkqgdbxPXz7LfBpA    ujmEBvifdJM6h6RLv4wQIg
                                   ...          
i7K3PDFJzXJSebzCj8UYJw    7BIH7ga7YotCM_Bh6qplIA
i7K3PDFJzXJSebzCj8UYJw    7BIH7ga7YotCM_Bh6qplIA
I3E0WQMyDhkvfrLHVot00w    lPgE2MIteQVzs39bwfmS6Q
vWUj8ed8ojO8CctKP-mwxQ    lPgE2MIteQVzs39bwfmS6Q
vWUj8ed8ojO8CctKP-mwxQ    lPgE2MIteQVzs39bwfmS6Q
Name: business_id, Length: 311191, dtype: object
0
